In [1]:
!pip install pypuf
import pypuf


In [2]:
#each drone has a secret shared with the GS

#we will take 100 challenge response pairs between the GS and drone to establish the secret key between them

#we have demonstrated the mechanism between 3 drone and GS

from pypuf.simulation import XORArbiterPUF

puf1 = XORArbiterPUF(n=128, k=10, seed=1) 
puf2=  XORArbiterPUF(n=128, k=12, seed=1)
puf3=  XORArbiterPUF(n=128, k=14, seed=1)


#similarly we will have puf2 and puf3 for drone 2 and drone 3 in a 3drone system

#get to know more about k and seed from here --> https://pypuf.readthedocs.io/en/latest/simulation/arbiter_puf.html

from pypuf.io import random_inputs

challenges1 = random_inputs(n= 128, N=128, seed=2)  #taking 128 challenges of 128bits
challenges2 = random_inputs(n= 128, N=128, seed=4)
challenges3 = random_inputs(n= 128, N=128, seed=6)


challenges1
#type(challenges)

array([[-1, -1,  1, ..., -1, -1, -1],
       [-1,  1, -1, ...,  1, -1,  1],
       [-1,  1,  1, ..., -1, -1,  1],
       ...,
       [ 1,  1,  1, ..., -1, -1,  1],
       [ 1,  1,  1, ..., -1,  1, -1],
       [ 1, -1, -1, ...,  1,  1, -1]], dtype=int8)

In [3]:
#now we are getting the response from the challenge which will act as a secret key between the GS and the drone

#so the key is of length 128
key1=puf1.eval(challenges1)
key2=puf2.eval(challenges2)
key3=puf3.eval(challenges3)
print(type(key1))
key1


<class 'numpy.ndarray'>


array([-1, -1,  1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,
        1,  1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
        1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,
        1,  1, -1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1, -1,  1,
        1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,
        1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1, -1, -1,  1, -1], dtype=int8)

In [4]:

import base64
import random

base64.b16encode(random.getrandbits(128).to_bytes(16, byteorder='little'))


b'C99677E5439393212FF4E55B151A907F'

In [5]:
#now we will assign id to each drone which will be a 128 bit key
import random
hash1 = random.getrandbits(128)  #we will assign this to drone 1
#print(int(hash))
hash2 = random.getrandbits(128) #this to drone 2
#print(int(hash2))
hash3 = random.getrandbits(128) #this to drone 3
#print(int(hash3))

arr=[] #declaring a list which will store the values of the IDs
arr.append(hash1)
arr.append(hash2)
arr.append(hash3)

print(arr)

arr.sort()

print(arr)



#sort(hash1,hash2,hash3)

[301877738346905656038643419403732757264, 321776331920994136706739406718215374755, 53231788789834811042400695555960636914]
[53231788789834811042400695555960636914, 301877738346905656038643419403732757264, 321776331920994136706739406718215374755]


In [6]:
#converting the list to string to append all the id's to calculate the hash

s=""
for i in arr:
  s=s+str(i)

print(s)
len(s)

53231788789834811042400695555960636914301877738346905656038643419403732757264321776331920994136706739406718215374755


116

In [7]:
#now we will try to generate the group name by passing the list to the hash function

# Python 3 code to demonstrate the
# working of MD5 (string - hexadecimal)

import hashlib


# encoding GeeksforGeeks using encode()
# then sending to md5()
result = hashlib.sha256(s.encode())

# printing the equivalent hexadecimal value.
#print("The hexadecimal equivalent of hash is : ", end ="")
res_in_hex= result.hexdigest()
print("This is the 256 bits group name--> " , res_in_hex)
#type(res_in_hex)


This is the 256 bits group name-->  e57e18461090a19eeaa94071c66d12692d675bf0489291e35e4ae3f7062419f3


In [8]:
#now we will generate a nonce just to append it with the group name and then hash it to get the group secret

#G_s= PUF of Ground Station (G_n||Nonce)

import secrets
nonce = secrets.token_hex(32)  #32byte=256bit nonce generator
print(nonce)
#type(nonce)

9a4972477cc26c9873a1b2ade6d5ccbd530b30142a10c6802b6345a17162820e


In [9]:
#now we will append the group name and the nonce

x=""
x=x+res_in_hex
x=x+nonce
x
#type(x)
len(x)

#this results in 96 bit hex hash code 

128

In [10]:
# Python code to demonstrate
# conversion of hex string
# to binary string

import math

# Initialising hex string
ini_string = x

# Printing initial string
print ("Initial string", ini_string)

# Code to convert hex to binary
res = "{0:08b}".format(int(ini_string, 16))

# Print the resultant string
print ("Resultant string", str(res))

print("the length is--> ", len(res)) 




Initial string e57e18461090a19eeaa94071c66d12692d675bf0489291e35e4ae3f7062419f39a4972477cc26c9873a1b2ade6d5ccbd530b30142a10c6802b6345a17162820e
Resultant string 11100101011111100001100001000110000100001001000010100001100111101110101010101001010000000111000111000110011011010001001001101001001011010110011101011011111100000100100010010010100100011110001101011110010010101110001111110111000001100010010000011001111100111001101001001001011100100100011101111100110000100110110010011000011100111010000110110010101011011110011011010101110011001011110101010011000010110011000000010100001010100001000011000110100000000010101101100011010001011010000101110001011000101000001000001110
the length is-->  512


In [13]:
#now to add the leading 0's so that it remains 512 bits

res1=""
for i in range(len(res) , 512):
  res1=res1+'0'

res1=res1+res

res=res1

len(res)


512

In [14]:
#now we will feed this to the PUF of ground station, for that we need to design the PUF


#we are declaring a PUF that will take 8 bit as input, so that we can have 64 bit output since (G_n||Nonce) is 512bits 
puf_gs = XORArbiterPUF(n=8, k=10, seed=1) 



In [15]:
#modifying the 256 response so that we can store in a 2D array

i=0
z=""
while(i<=len(res)):
  if(i!=0):
    if(i%8!=0):
      z=z+res[i-1:i]+"\t"
    else:
      z=z=z+res[i-1:i]+"\n"

  i+=1

z



'1\t1\t1\t0\t0\t1\t0\t1\n0\t1\t1\t1\t1\t1\t1\t0\n0\t0\t0\t1\t1\t0\t0\t0\n0\t1\t0\t0\t0\t1\t1\t0\n0\t0\t0\t1\t0\t0\t0\t0\n1\t0\t0\t1\t0\t0\t0\t0\n1\t0\t1\t0\t0\t0\t0\t1\n1\t0\t0\t1\t1\t1\t1\t0\n1\t1\t1\t0\t1\t0\t1\t0\n1\t0\t1\t0\t1\t0\t0\t1\n0\t1\t0\t0\t0\t0\t0\t0\n0\t1\t1\t1\t0\t0\t0\t1\n1\t1\t0\t0\t0\t1\t1\t0\n0\t1\t1\t0\t1\t1\t0\t1\n0\t0\t0\t1\t0\t0\t1\t0\n0\t1\t1\t0\t1\t0\t0\t1\n0\t0\t1\t0\t1\t1\t0\t1\n0\t1\t1\t0\t0\t1\t1\t1\n0\t1\t0\t1\t1\t0\t1\t1\n1\t1\t1\t1\t0\t0\t0\t0\n0\t1\t0\t0\t1\t0\t0\t0\n1\t0\t0\t1\t0\t0\t1\t0\n1\t0\t0\t1\t0\t0\t0\t1\n1\t1\t1\t0\t0\t0\t1\t1\n0\t1\t0\t1\t1\t1\t1\t0\n0\t1\t0\t0\t1\t0\t1\t0\n1\t1\t1\t0\t0\t0\t1\t1\n1\t1\t1\t1\t0\t1\t1\t1\n0\t0\t0\t0\t0\t1\t1\t0\n0\t0\t1\t0\t0\t1\t0\t0\n0\t0\t0\t1\t1\t0\t0\t1\n1\t1\t1\t1\t0\t0\t1\t1\n1\t0\t0\t1\t1\t0\t1\t0\n0\t1\t0\t0\t1\t0\t0\t1\n0\t1\t1\t1\t0\t0\t1\t0\n0\t1\t0\t0\t0\t1\t1\t1\n0\t1\t1\t1\t1\t1\t0\t0\n1\t1\t0\t0\t0\t0\t1\t0\n0\t1\t1\t0\t1\t1\t0\t0\n1\t0\t0\t1\t1\t0\t0\t0\n0\t1\t1\t1\t0\t0\t1\t1\n1\t0\t1\t0\t0\t

In [16]:
import numpy as np

a = np.array([[int(j) for j in i.split('\t')] for i in z.splitlines()])
a


array([[1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 0, 1, 0],
       [1, 1, 1, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0,

In [17]:
#now we need to convert the 0's to -1 to feed it as an input to the PUF

for i in range(len(a)):
  for j in range(len(a[i])):
    if(a[i][j]==0):
      a[i][j]=-1

a

array([[ 1,  1,  1, -1, -1,  1, -1,  1],
       [-1,  1,  1,  1,  1,  1,  1, -1],
       [-1, -1, -1,  1,  1, -1, -1, -1],
       [-1,  1, -1, -1, -1,  1,  1, -1],
       [-1, -1, -1,  1, -1, -1, -1, -1],
       [ 1, -1, -1,  1, -1, -1, -1, -1],
       [ 1, -1,  1, -1, -1, -1, -1,  1],
       [ 1, -1, -1,  1,  1,  1,  1, -1],
       [ 1,  1,  1, -1,  1, -1,  1, -1],
       [ 1, -1,  1, -1,  1, -1, -1,  1],
       [-1,  1, -1, -1, -1, -1, -1, -1],
       [-1,  1,  1,  1, -1, -1, -1,  1],
       [ 1,  1, -1, -1, -1,  1,  1, -1],
       [-1,  1,  1, -1,  1,  1, -1,  1],
       [-1, -1, -1,  1, -1, -1,  1, -1],
       [-1,  1,  1, -1,  1, -1, -1,  1],
       [-1, -1,  1, -1,  1,  1, -1,  1],
       [-1,  1,  1, -1, -1,  1,  1,  1],
       [-1,  1, -1,  1,  1, -1,  1,  1],
       [ 1,  1,  1,  1, -1, -1, -1, -1],
       [-1,  1, -1, -1,  1, -1, -1, -1],
       [ 1, -1, -1,  1, -1, -1,  1, -1],
       [ 1, -1, -1,  1, -1, -1, -1,  1],
       [ 1,  1,  1, -1, -1, -1,  1,  1],
       [-1,  1, 

In [18]:
#now we will feed this to the puf of gs
group_secret= puf_gs.eval(a)

group_secret

array([-1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1,
        1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1,
       -1,  1, -1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1])

In [19]:
#now we will convert the group secret to a string so that it can be fed as input to the AES256 bit encryption
st_gs=""


for i in range(len(group_secret)):
  if group_secret[i]==-1 :
    st_gs=st_gs+str(0)
  else :
    st_gs=st_gs+str(group_secret[i])


st_gs



'0110001111110000000100110111101100110111111000010110100001011111'

In [20]:
len(st_gs)

64

###**We need to perform encryption using the PUF result of the drone id hence so we are coverting the key to a bit stream and then we will convert it to hex to use it as the password of AES128**

In [21]:
#changing the key from 1 and -1 format to 1 and 0 format

for i in range(len(key1)):
  if(key1[i]==-1):
    key1[i]=0


for i in range(len(key2)):
  if(key2[i]==-1):
    key2[i]=0


for i in range(len(key3)):
  if(key3[i]==-1):
    key3[i]=0


key3

array([1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], dtype=int8)

In [23]:
#converting the key to string of bits

st1=""
st2=""
st3=""

for i in range(len(key1)):
  val=int(key1[i])
  st1=st1+str(val)


for i in range(len(key2)):
  val=int(key2[i])
  st2=st2+str(val)


for i in range(len(key3)):
  val=int(key3[i])
  st3=st3+str(val)


st3


'11100100110101011101001110000111110011011000001010101011011011111001100010000100001001010010000001111110111010011101010010010001'

In [24]:
#converting the string to hex format

dec_str = format(int(st1, 2),'x')
pass1=dec_str.rjust(32,'0')
pass1


dec_str = format(int(st2, 2),'x')
pass2=dec_str.rjust(32,'0')
pass2


dec_str = format(int(st3, 2),'x')
pass3=dec_str.rjust(32,'0')
pass3


#this pass1 will be fed as the password of AES256

'e4d5d387cd82ab6f988425207ee9d491'

In [25]:
!pip install pycryptodomex

     |████████████████████████████████| 2.0 MB 5.1 MB/s 


In [26]:
!pip install pycrypto

     |████████████████████████████████| 446 kB 4.7 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=dc5eeb1d8d08f9274369fcf2bed8973fa7756688f9edec2f2986a3b472dbb210
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


In [27]:
#AES 128 

from Crypto.Cipher import AES
 
key1 = pass1
 
cipher = AES.new(key1, AES.MODE_ECB)
msg1 =cipher.encrypt(st_gs)
print (type(msg1))
 
print(msg1)
 
decipher = AES.new(key1, AES.MODE_ECB)
print(decipher.decrypt(msg1))

<class 'bytes'>
b'\x95\xe0*\xb9\xa3}8\xea\xf9\x946\x94\xf5\x86\xd3.iks\xe9\xa86\x13\x1c\xb4\xff\x93\xbb"s\x80\xc4\xca\xe7r\x8f\\P\xfd\xf3hA\xddV\x16h\xe5p\x8e\x00\x99@\xdd\xbd\xe4\xc6\x80\x1d\xd2l\xb1N\x1e\xca'
b'0110001111110000000100110111101100110111111000010110100001011111'


In [28]:
#AES 128 

from Crypto.Cipher import AES
 
key2 = pass2
 
cipher = AES.new(key2, AES.MODE_ECB)
msg2 =cipher.encrypt(st_gs)
print (type(msg2))
 
print(msg2)
 
decipher = AES.new(key2, AES.MODE_ECB)
print(decipher.decrypt(msg2))

<class 'bytes'>
b'(\xfe\x19\xac\x0ekPx\x1a\xd0\xfc\x98cJP\xe3\xd9R\xd1\xaa\x16Z\r\xfc\xf0\x0c],\xbe\x8b\xf5\xe5a}E\xc2f\x899\x03B\xc0\xe7\xde\xe6\t\x8e)wa\x02\xa7\xd4\xbd\x0f\xea+\xffM\x8d\x91J\xd5\xe6'
b'0110001111110000000100110111101100110111111000010110100001011111'


In [29]:
#AES 128 

from Crypto.Cipher import AES
 
key3 = pass3
 
cipher = AES.new(key3, AES.MODE_ECB)
msg3 =cipher.encrypt(st_gs)
print (type(msg3))
 
print(msg3)
 
decipher = AES.new(key3, AES.MODE_ECB)
print(decipher.decrypt(msg3))

<class 'bytes'>
b'\x1f(\x17\x02\x11.\x15D\xc0\x00`B\x18\xb4Z\x85\x9d\xe0\xd8=\xc7\x1d\xc9i\xa5\xf9\xd4\x85\x14\x9e\x92\x92\xfd\x88\xb6\xfa@!\x95K\x86K\x05\x8f\xbas\xa0&\xef\x12\xd7H\xdb\xfe\xffl\n\xc9\xd5\x12\x14\xff\xbd\x8d'
b'0110001111110000000100110111101100110111111000010110100001011111'
